In [1]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
import ipywidgets
from IPython.display import display

# Camera and Motor Interface for JetBot
from jetbot import Robot, Camera, bgr8_to_jpeg

# Python basic pakcages for image annotation
import json
import glob
import time
import numpy as np
import cv2
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import PIL.Image

In [2]:
camera = Camera()
robot = Robot()

In [3]:
image_widget = widgets.Image(format='jpeg', width=224, height=224)
target_widget = widgets.Image(format='jpeg', width=224, height=224)

x_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='x')
y_slider = widgets.FloatSlider(min=-1.0, max=1.0, step=0.001, description='y')

def display_xy(camera_image):
    image = np.copy(camera_image)
    x = x_slider.value
    y = y_slider.value
    x = int(x * 224 / 2 + 112)
    y = int(y * 224 / 2 + 112)
    image = cv2.circle(image, (x, y), 8, (0, 255, 0), 3)
    image = cv2.circle(image, (112, 224), 8, (0, 0,255), 3)
    image = cv2.line(image, (x,y), (112,224), (255,0,0), 3)
    jpeg_image = bgr8_to_jpeg(image)
    return jpeg_image

time.sleep(5)

traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)
traitlets.dlink((camera, 'value'), (target_widget, 'value'), transform=display_xy)

In [4]:
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2)
MODEL_PATH = 'model_trail_03.pth'

model.load_state_dict(torch.load(MODEL_PATH))

device = torch.device('cuda')
model = model.to(device)
model = model.eval().half()

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.3, step=0.01, value=0.08, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=0.1, step=0.01, value=0.02, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.08, description='speed gain', max=0.3, step=0.01)

FloatSlider(value=0.02, description='steering gain', max=0.1, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

In [6]:
angle = 0.0
angle_last = 0.0

def execute(change):
    global angle, angle_last
    image = change['new']
    xy = model(preprocess(image)).detach().float().cpu().numpy().flatten()
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0
    
    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    robot.left_motor.value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    robot.right_motor.value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)
    
execute({'new': camera.value})

In [7]:
# create buttons
button_layout = widgets.Layout(width='100px', height='80px', align_self='center')
stop_button = widgets.Button(description='stop', button_style='danger', layout=button_layout)
forward_button = widgets.Button(description='forward', layout=button_layout)
backward_button = widgets.Button(description='backward', layout=button_layout)
left_button = widgets.Button(description='left', layout=button_layout)
right_button = widgets.Button(description='right', layout=button_layout)
start_bot_button = widgets.Button(description='start bot', button_style='success', layout=button_layout)
stop_bot_button = widgets.Button(description='stop bot', button_style='danger', layout=button_layout)

# define buttons and actions
def stop(change):
    robot.stop()
    
def step_forward(change):
    robot.forward(0.2)
    time.sleep(0.5)
    robot.stop()

def step_backward(change):
    robot.backward(0.2)
    time.sleep(0.5)
    robot.stop()

def step_left(change):
    robot.left(0.1)
    time.sleep(0.4)
    robot.stop()

def step_right(change):
    robot.right(0.1)
    time.sleep(0.4)
    robot.stop()

def start_bot(change):   
    camera.observe(execute, names='value')
    
def stop_bot(change):    
    camera.unobserve(execute, names='value')
    robot.stop()

# link buttons to actions
stop_button.on_click(stop)
forward_button.on_click(step_forward)
backward_button.on_click(step_backward)
left_button.on_click(step_left)
right_button.on_click(step_right)
start_bot_button.on_click(start_bot)
stop_bot_button.on_click(stop_bot)

# display buttons
top_box = widgets.HBox([start_bot_button, forward_button, start_bot_button], layout=widgets.Layout(align_self='center'))
middle_box = widgets.HBox([left_button, backward_button, right_button], layout=widgets.Layout(align_self='center'))
bottom_box = widgets.HBox([stop_bot_button, stop_button, stop_bot_button], layout=widgets.Layout(align_self='center')) 

In [8]:
display(widgets.HBox([image_widget, target_widget]))

In [9]:
display(top_box)
display(middle_box)
display(bottom_box)

In [10]:
display(ipywidgets.HBox([y_slider, speed_slider]))